In [16]:
import pandas as pd
from citipy import citipy
import csv
import quandl
from api import api_key


ModuleNotFoundError: No module named 'api'

In [2]:
airbnb = pd.read_csv("../../train.csv")


In [3]:
airbnb.head(20)
lat = airbnb['latitude']
lon = airbnb['longitude']
cities = []
state=[]

lat_lng = zip(lat, lon)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lng:
    
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    cities.append(city)

# Print the city count to confirm sufficient count
print(city)

long beach


In [4]:
airbnb['cities'] = cities
airbnb['cities'].value_counts()

new york               20344
san francisco           5505
west hollywood          5144
edgewater               5076
washington              4100
                       ...  
elizabeth                  1
calumet city               1
walnut park                1
north valley stream        1
tiburon                    1
Name: cities, Length: 164, dtype: int64

In [5]:
airbnb['city'].value_counts()

NYC        32349
LA         22453
SF          6434
DC          5688
Chicago     3719
Boston      3468
Name: city, dtype: int64

In [6]:
airbnb.columns

Index(['id', 'log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'city', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'last_review', 'latitude',
       'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode', 'bedrooms', 'beds',
       'cities'],
      dtype='object')

In [7]:
airbnb_new=airbnb[['id','city', 'neighbourhood', 'latitude', 'longitude', 'first_review']]

In [8]:
airbnb_new.to_csv("Train reduced.csv", index=False)